In [1]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [2]:
#Read data from csv
train_data = pd.read_csv('fmnist/fashion-mnist_train.csv')
test_data = pd.read_csv('fmnist/fashion-mnist_test.csv')

In [3]:
#Setting images parameters
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

X = np.array(train_data.iloc[:, 1:])
y = to_categorical(np.array(train_data.iloc[:,0]))

In [9]:
#Separating train data into train and validation set
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.2, random_state=13)

In [20]:
#Load test data into X_test and Y_test
X_test = np.array(test_data.iloc[:, 1:])
Y_test = to_categorical(np.array(test_data.iloc[:, 0]))

#Now preprocessing of data
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

X_train /= 255
X_test /= 255
X_val /=255

In [21]:
#Building our CNN using keras

batch_size = 256
n_classes = 10
epochs = 50

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 128)         73856     
__________

In [23]:
#Evaluation of our model
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val,Y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 21s - loss: 0.8831 - acc: 0.6664 - val_loss: 0.5333 - val_acc: 0.8066
Epoch 2/50
48000/48000 [==============================] - 21s - loss: 0.5409 - acc: 0.7964 - val_loss: 0.4388 - val_acc: 0.8376
Epoch 3/50
48000/48000 [==============================] - 21s - loss: 0.4675 - acc: 0.8274 - val_loss: 0.3998 - val_acc: 0.8549
Epoch 4/50
48000/48000 [==============================] - 21s - loss: 0.4207 - acc: 0.8438 - val_loss: 0.3566 - val_acc: 0.8675
Epoch 5/50
48000/48000 [==============================] - 21s - loss: 0.3897 - acc: 0.8549 - val_loss: 0.3449 - val_acc: 0.8698
Epoch 6/50
48000/48000 [==============================] - 20s - loss: 0.3638 - acc: 0.8657 - val_loss: 0.3211 - val_acc: 0.8817
Epoch 7/50
48000/48000 [==============================] - 21s - loss: 0.3467 - acc: 0.8743 - val_loss: 0.3016 - val_acc: 0.8900
Epoch 8/50
48000/48000 [==============================

In [24]:
#Accuracy on testing data
score = model.evaluate(X_test, Y_test, verbose=0)

In [32]:
#Print testing data loss and accuracy of our model
print('Loss:',score[0])
print('Accuracy:',score[1])

Loss: 0.202474860071
Accuracy: 0.9277


In [33]:
#save model
model.save('fashion-mnist-cnn.json')
model.save_weights('fashion-mnist-weights.h5')